# Send REST requests to the API running in Aspire

In [ ]:
var baseUrl = "https://localhost:7220";

## Healthcheck 💉

In [2]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/

Name,Value
traceparent,00-bfa3be59032cea6c6d71ceb512cd1cab-256b60d7ab9b3195-00
Name,Value
Date,"Sat, 19 Apr 2025 11:01:00 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,text/plain; charset=utf-8
Content-Length,26


## GET Weather Forecast 🌦️🌈🔆

In [3]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/v1/weather-forecast/Bristol/{{$datetime iso8601}}

Name,Value
traceparent,00-212842a975c0dfdbe6b78cebc438a4d7-d835fb294bf5f5ef-00
Name,Value
Date,"Sat, 19 Apr 2025 11:01:16 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,175
requestedRegion,"""Bristol"""
requestedDate,"""2025-04-19T11:01:16.9197185Z"""
requestId,"""8afa3050-d5f2-4efc-a8b5-d2fb29aebe6f"""


## POST Colected Weather Data 🌡️☔💨

In [5]:
// Generate a random reference...
string[] fruits = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "watermelon", "guava"];
var randomReference = $"{fruits[new Random().Next(fruits.Length)]}-{Random.Shared.Next(10, 99)}";
randomReference.Display();

tangerine-81

In [7]:
#!set --value @csharp:baseUrl --name baseUrl
#!set --value @csharp:randomReference --name randomReference

POST {{baseUrl}}/v1/collected-weather-data/Bristol/{{randomReference}}
Content-Type: application/json
    
{
    "points": [
        {
            "time": "{{$datetime iso8601}}",
            "windSpeedInMetersPerSecond": {{$randomInt 0 69}}.{{$randomInt 0 99}},
            "windDirection": "N",
            "temperatureReadingInDegreesCelcius": {{$randomInt -15 40}}.{{$randomInt 0 99}},
            "humidityReadingInPercent": {{$randomInt 20 100}}
        }
    ]
}

Name,Value
traceparent,00-903a7eed3e548e0bd2abf436ffdc979b-ae06d346ba82491a-00
Content-Type,application/json
Content-Length,279
Name,Value
Date,"Sat, 19 Apr 2025 11:01:45 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,79
requestId,"""14adcc26-f540-412d-9d45-9fe9b2e8966f"""


<div style="display:block; height:40rem;"></div>

## GET Traces 🤖

In [25]:
using System.Net.Http;
using System.Text.Json;

public class TraceData
{
    public string Resource { get; set; } = string.Empty;
    public string Source { get; set; } = string.Empty;
    public string DisplayName { get; set; } = string.Empty;
    public string TraceId { get; set; } = string.Empty;
    public string SpanId { get; set; } = string.Empty;    
    public Dictionary<string, object?> Tags { get; set; } = new();

    public override string ToString()
    {
        var thirdPart = string.Empty;
        if(DisplayName == "Domain Event Insertion")
            thirdPart = Tags["domain-event.eventclassName"].ToString().Split('.').LastOrDefault();

        if(DisplayName == "User Notication Sent")
            thirdPart = $"✉️ reference: {Tags["user-notification-event.reference"].ToString()}";

        if(DisplayName == "Outbox Item Insertion")
            thirdPart = $"📫 outbox id: {Tags["outbox-item.Id"].ToString()}";
        
            
        
        return $"{Resource.PadRight(30, '-')}> {DisplayName.PadRight(30, '-')}> {thirdPart}";
    } 
        
}

var client = new HttpClient() { BaseAddress = new Uri("https://localhost:7059") };
var response = await client.GetAsync("traces");
var responseJson = await response.Content.ReadAsStringAsync();
JsonSerializer.Deserialize<TraceData[]>(responseJson, JsonSerializerOptions.Web ).Display();

index value 0 NotificationService-----------> User Notication Sent----------> ✉️ reference: tangerine-81 Resource NotificationService Source UserNotificationEventHandler DisplayName User Notication Sent TraceId 903a7eed3e548e0bd2abf436ffdc979b SpanId 629a7f2af993e503 Tags key value user-notification-event.body "Dear user, your data has been submitted and included in our latest model" user-notification-event.reference "tangerine-81" user-notification-event.timestamp "2025-04-19T11:01:49.3538419+00:00" 1 WeatherApp.EventListener------> Domain Event Insertion--------> ModelingDataAccepted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 903a7eed3e548e0bd2abf436ffdc979b SpanId 279d691f134022a6 Tags key value domain-event.streamId "14adcc26-f540-412d-9d45-9fe9b2e8966f" domain-event.version "5" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelingDataAccepted" 2 WeatherApp.API----------------> Domain Event Insertion--------> LocationIdFound Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 903a7eed3e548e0bd2abf436ffdc979b SpanId bffc0380b155eb48 Tags key value domain-event.streamId "14adcc26-f540-412d-9d45-9fe9b2e8966f" domain-event.version "2" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.LocationIdFound" 3 WeatherApp.EventListener------> Outbox Item Insertion---------> 📫 outbox id: 10061 Resource WeatherApp.EventListener Source OutboxRepository DisplayName Outbox Item Insertion TraceId 903a7eed3e548e0bd2abf436ffdc979b SpanId 11a4303d3e749b7e Tags key value outbox-item.TypeName "UserNotificationEvent" outbox-item.Id 10061 4 WeatherApp.EventListener------> Domain Event Insertion--------> ModelUpdated Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 903a7eed3e548e0bd2abf436ffdc979b SpanId 24502e1dd12f4919 Tags key value domain-event.streamId "14adcc26-f540-412d-9d45-9fe9b2e8966f" domain-event.version "8" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelUpdated" 5 WeatherApp.EventListener------> Domain Event Insertion--------> SubmissionComplete Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 903a7eed3e548e0bd2abf436ffdc979b SpanId 2a1476898884964d Tags key value domain-event.streamId "14adcc26-f540-412d-9d45-9fe9b2e8966f" domain-event.version "7" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmissionComplete" 6 WeatherApp.EventListener------> Domain Event Insertion--------> PendingContributorPaymentCommitted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 903a7eed3e548e0bd2abf436ffdc979b SpanId 4093092fd914201d Tags key value domain-event.streamId "14adcc26-f540-412d-9d45-9fe9b2e8966f" domain-event.version "6" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentCommitted" 7 WeatherApp.API----------------> Domain Event Insertion--------> SubmittedToModeling Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 903a7eed3e548e0bd2abf436ffdc979b SpanId 445ea4e0be4f9314 Tags key value domain-event.streamId "14adcc26-f540-412d-9d45-9fe9b2e8966f" domain-event.version "4" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmittedToModeling" 8 WeatherApp.API----------------> Domain Event Insertion--------> PendingContributorPaymentPosted Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 903a7eed3e548e0bd2abf436ffdc979b SpanId 118481e673859283 Tags key value domain-event.streamId "14adcc26-f540-412d-9d45-9fe9b2e8966f" domain-event.version "3" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentPosted" 9 WeatherApp.API----------------> Domain Event Insertion--------> WeatherDataCollectionInitiated Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event I